In [1]:
from transformers import AutoModelForSequenceClassification , AutoModelForCausalLM
import torch

In [2]:
import os 
FILE_PATH = os.path.dirname(os.path.abspath(""))
FILE_PATH

'/Users/mohitdulani/Desktop/personal/rlhf'

### Visualise causal LM weights 

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

BASE_MODEL = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, cache_dir = FILE_PATH)
lm_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, cache_dir = FILE_PATH)
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(lm_model.device)

# outputs = model.generate(**inputs, max_new_tokens=40)
# print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))


In [ ]:
dir(lm_model)

In [ ]:
for param in lm_model.parameters():
    param.requires_grad = False

for param in lm_model.lm_head.parameters():
    param.requires_grad = True

# lm_model.state_dict().items()

In [ ]:
lm_model.model

In [ ]:
lm_model.lm_head.in_features

In [ ]:
aa= inputs.get('input_ids') 
print(aa.size())
torch.cat([aa , torch.Tensor(data=[[32]])], dim =1)


In [ ]:
import torch 
outs = []

inps = inputs.get('input_ids')
for _ in range(100):
    out_toks = lm_model.model.embed_tokens(inps)
    print(out_toks.shape)
    # model_in = lm_model.model.layers(out_toks)

    hidden = out_toks
    for layer in lm_model.model.layers: 
        print('inside list')
        hidden = layer(hidden)
        print("hidden is ", hidden)
        hidden = hidden[0]

    out_linear = lm_model.lm_head(hidden)
    print(out_linear.shape)
    findmax = torch.argmax(out_linear, dim=-1)
    add_back = findmax[:,-1].item()
    outs.append(add_back)
    print(add_back)
    print(inps)
    print(type(inps))
    inps = torch.cat([inps, torch.tensor([[add_back]], dtype=inps.dtype)], dim=1)

outs

In [ ]:
import torch 
findmax = torch.argmax(out_linear, dim=-1)
findmax[:,-1].item()


In [ ]:
decoded = tokenizer.batch_decode(findmax, skip_special_tokens=True)
decoded

In [ ]:
out_layer = lm_model.model(inputs.get('input_ids'))
out_layer

In [ ]:
out_layer.last_hidden_state.shape

### Visualise sequence classification model 

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

BASE_MODEL = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, cache_dir = FILE_PATH)
seq_model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, cache_dir = FILE_PATH, num_labels = 5)

text = "I am cool"

# Tokenize (no chat template needed)
inputs = tokenizer(text, return_tensors="pt").to(seq_model.device)

# Forward pass
with torch.no_grad():
    outputs = seq_model(**inputs)
    logits = outputs.logits
    probs = torch.softmax(logits, dim=-1)
    pred_class = torch.argmax(probs, dim=-1).item()

print("Probabilities:", probs)
print("Predicted class:", pred_class)

# outputs = model.generate(**inputs, max_new_tokens=40)
# print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))


In [ ]:
dir(seq_model) , seq_model.__class__, seq_model.generation_config , seq_model.from_pretrained, seq_model.forward

In [ ]:

text = "this was a terrific movie  "


# Tokenize (no chat template needed)
inputs = tokenizer(text, return_tensors="pt").to(seq_model.device)

# Forward pass
with torch.no_grad():
    outputs = seq_model(**inputs)
    logits = outputs.logits
    probs = torch.softmax(logits, dim=-1)
    pred_class = torch.argmax(probs, dim=-1).item()

print("Probabilities:", probs)
print("Predicted class:", pred_class)


In [ ]:
seq_model.state_dict

### Visualizing configs


In [ ]:
from transformers import AutoConfig
BASE_MODEL = "google/gemma-2b-it"

config = AutoConfig.from_pretrained(BASE_MODEL) 

config

### Using the converted model 


In [ ]:
from conversion import ConvertedModel
conv_model = ConvertedModel()

In [ ]:
# use trainer class /  reward model in this 
from transformers import HfArgumentParser
from datasets import load_dataset
from conversion import ConvertedModel
conv_model = ConvertedModel()
from trl import (
    ModelConfig,
    RewardConfig,
    RewardTrainer,
    ScriptArguments,
    get_kbit_device_map,
    get_peft_config,
    get_quantization_config,
    setup_chat_format,
)


# parser = HfArgumentParser((ScriptArguments, RewardConfig, ModelConfig))
# script_args, training_args, model_args = parser.parse_args_into_dataclasses(args = [])
# training_args.gradient_checkpointing_kwargs = dict(use_reentrant=False)

dataset = load_dataset(path ='Dahoas/rm-static', cache_dir = FILE_PATH)

# trainer = RewardTrainer(
#     model=conv_model,
#     processing_class=tokenizer,
#     train_dataset=dataset
# )


reward_args = RewardConfig(
    output_dir=FILE_PATH,
    per_device_train_batch_size=2,
    num_train_epochs=1,
)

trainer = RewardTrainer(
    model=conv_model,
    processing_class=tokenizer,
    args=reward_args,
    # train_dataset=dataset[script_args.dataset_train_split],
    # eval_dataset=dataset[script_args.dataset_test_split] if training_args.eval_strategy != "no" else None,
    
    # peft_config=get_peft_config(model_args),

)



In [ ]:

from transformers import AutoConfig
from conversion import ConvertedModel

config = AutoConfig.from_pretrained("google/gemma-3-270m-it", num_labels=1, cache_dir = FILE_PATH)

print("config is ", config)

conv_model = ConvertedModel(config)

In [ ]:
from transformers import PreTrainedModel, AutoConfig, AutoModelForCausalLM
import torch.nn as nn

class ConvertedModel2(PreTrainedModel):
    def __init__(self, config, model_name="google/gemma-3-270m-it", num_labels=1, **kwargs):
        super().__init__(config)
        self.model_name = model_name
        self.num_labels = num_labels

        self.lm_model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir= FILE_PATH)
        for p in self.lm_model.parameters():
            p.requires_grad = False

        logits_count = self.lm_model.lm_head.in_features
        self.lm_model.lm_head = nn.Linear(logits_count, num_labels)
        # self.classifier = self.lm_model.lm_head

    def forward(self, input_ids, attention_mask=None, labels=None, **kwargs):
        outputs = self.lm_model.model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden = outputs.last_hidden_state
        pooled = last_hidden[:, -1, :]
        # logits = self.classifier(pooled)
        logits = self.lm_model.lm_head(pooled)

        return {"logits": logits}

# usage
from transformers import AutoConfig
config = AutoConfig.from_pretrained("google/gemma-3-270m-it")
conv_model = ConvertedModel2(config)


In [ ]:
conv_model

In [ ]:
import wandb
wandb.login()


In [ ]:
wandb.init(
    project="rlhf-reward-model",
    name="gemma_linear_head",
)

In [ ]:
# use trainer class /  reward model in this 
from transformers import HfArgumentParser
from datasets import load_dataset
# from conversion import ConvertedModel
# conv_model = ConvertedModel()
from trl import (
    ModelConfig,
    RewardConfig,
    RewardTrainer,
    ScriptArguments,
    get_kbit_device_map,
    get_peft_config,
    get_quantization_config,
    setup_chat_format,
)


dataset = load_dataset(path ='Dahoas/rm-static', cache_dir = FILE_PATH)

# reward_args = RewardConfig(
#     output_dir=FILE_PATH,
#     do_train = True,
#     do_eval = True,  
#     per_device_train_batch_size=2,
#     num_train_epochs=1,
#     logging_steps = 10, 
# )


reward_args = RewardConfig(
    output_dir=os.path.join(FILE_PATH, "reward_outputs"),   # checkpoint + logs dir
    logging_dir=os.path.join(FILE_PATH, "logs"),            # local tensorboard logs
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=2,
    num_train_epochs=10,
    logging_steps=10,
    gradient_checkpointing=False, 

    # Precision
    fp16=False,          # disable mixed precision
    bf16=False,

    # Checkpoint saving
    save_strategy="steps",   # instead of "epoch"
    save_steps=5,            # save every 5 steps
    save_total_limit=2,      # keep only last 2 checkpoints

    # W&B integration
    report_to=[],       # logs training metrics to wandb
    run_name="reward_model_gemma_linear_head"  # nice display name in wandb
)

trainer = RewardTrainer(
    model=conv_model,
    processing_class=tokenizer,
    args=reward_args,
    train_dataset=dataset['train'], 
    eval_dataset = dataset['test'],

)

trainer.train()

In [ ]:
dataset['test'][:10]

In [ ]:
from conversion import ConvertedModel
from transformers import AutoConfig
import inspect
config = AutoConfig.from_pretrained("google/gemma-3-270m-it")
print(inspect.signature(ConvertedModel.__init__))
conv_model = ConvertedModel(config)

conv_model

#### Training a reward modelling 

In [ ]:
import os

import torch
from accelerate import logging
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, HfArgumentParser

from trl import (
    ModelConfig,
    RewardConfig,
    RewardTrainer,
    ScriptArguments,
    get_kbit_device_map,
    get_peft_config,
    get_quantization_config,
    setup_chat_format,
)


logger = logging.get_logger(__name__)

# Enable logging in a Hugging Face Space
os.environ.setdefault("TRACKIO_SPACE_ID", "trl-trackio")


if __name__ == "__main__":
    parser = HfArgumentParser((ScriptArguments, RewardConfig, ModelConfig))
    script_args, training_args, model_args = parser.parse_args_into_dataclasses()
    training_args.gradient_checkpointing_kwargs = dict(use_reentrant=False)

    ################
    # Model & Tokenizer
    ################
    torch_dtype = (
        model_args.torch_dtype if model_args.torch_dtype in ["auto", None] else getattr(torch, model_args.torch_dtype)
    )
    quantization_config = get_quantization_config(model_args)
    model_kwargs = dict(
        revision=model_args.model_revision,
        device_map=get_kbit_device_map() if quantization_config is not None else None,
        quantization_config=quantization_config,
        use_cache=False if training_args.gradient_checkpointing else True,
        torch_dtype=torch_dtype,
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_args.model_name_or_path, trust_remote_code=model_args.trust_remote_code, use_fast=True
    )
    model = AutoModelForSequenceClassification.from_pretrained(
        model_args.model_name_or_path, num_labels=1, trust_remote_code=model_args.trust_remote_code, **model_kwargs
    )
    # Align padding tokens between tokenizer and model
    model.config.pad_token_id = tokenizer.pad_token_id

    # If post-training a base model, use ChatML as the default template
    if tokenizer.chat_template is None:
        model, tokenizer = setup_chat_format(model, tokenizer)

    if model_args.use_peft and model_args.lora_task_type != "SEQ_CLS":
        logger.warning(
            "You are using a `task_type` that is different than `SEQ_CLS` for PEFT. This will lead to silent bugs"
            " Make sure to pass --lora_task_type SEQ_CLS when using this script with PEFT.",
        )

    ##############
    # Load dataset
    ##############
    dataset = load_dataset(script_args.dataset_name, name=script_args.dataset_config)

    ##########
    # Training
    ##########
    trainer = RewardTrainer(
        model=model,
        processing_class=tokenizer,
        args=training_args,
        train_dataset=dataset[script_args.dataset_train_split],
        eval_dataset=dataset[script_args.dataset_test_split] if training_args.eval_strategy != "no" else None,
        peft_config=get_peft_config(model_args),
    )
    trainer.train()

    ############################
    # Save model and push to Hub
    ############################
    trainer.save_model(training_args.output_dir)

    if training_args.eval_strategy != "no":
        metrics = trainer.evaluate()
        trainer.log_metrics("eval", metrics)
        trainer.save_metrics("eval", metrics)

    # Save and push to hub
    trainer.save_model(training_args.output_dir)
    if training_args.push_to_hub:
        trainer.push_to_hub(dataset_name=script_args.dataset_name)

## Training the model  

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM


BASE_MODEL = "google/gemma-3-270m-it"
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, cache_dir = FILE_PATH)
lm_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, cache_dir = FILE_PATH)

In [ ]:
# this files helps to convert the model from causal to sequence modelling part 
import torch 
import torch.nn as nn

import os
from transformers import AutoModelForSequenceClassification, AutoModelForCausalLM, PreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput

# take the base class as pretrained model 
class ConvertedModel(PreTrainedModel):
    def __init__(self, config, **kwargs):
        super().__init__(config)
        num_labels = kwargs.get('num_labels', 1)
        model = kwargs.get('model', "google/gemma-3-270m-it")
        self.model_name = model
        self.num_labels = num_labels

        self.FILE_PATH = FILE_PATH

        # Device selection (MPS > CUDA > CPU)
        if torch.backends.mps.is_available():
            self.device_str = 'mps'
        elif torch.cuda.is_available():
            self.device_str = 'cuda'
        else:
            self.device_str = 'cpu'

        # Load the base CausalLM model
        self.lm_model = AutoModelForCausalLM.from_pretrained(self.model_name, cache_dir=self.FILE_PATH)
        for params in self.lm_model.parameters():
            params.requires_grad = False

        # Replace the language modeling head with a classification head
        logits_count = self.lm_model.lm_head.in_features
        self.lm_model.lm_head = nn.Linear(
            in_features=logits_count,
            out_features=self.num_labels,
            bias=True,
            device=self.device_str
        )
        for params in self.lm_model.lm_head.parameters():
            params.requires_grad = True



    # def forward(self, input_ids, attention_mask=None):
    #     # Get hidden states from the base model
    #     outputs = self.lm_model.model(
    #         input_ids=input_ids,
    #         attention_mask=attention_mask,
    #         output_hidden_states=False
    #     )

    #     # Take last token representation for classification
    #     last_hidden = outputs.last_hidden_state  # [batch, seq_len, hidden_dim]
    #     pooled = last_hidden[:, -1, :]  # last token

    #     # Classify using the replaced head
    #     logits = self.lm_model.lm_head(pooled)
    #     return {"logits": logits}



    # def forward(
    #     self,
    #     input_ids=None,
    #     attention_mask=None,
    #     labels=None,
    #     return_dict=True,
    #     **kwargs
    # ):
    #     # Hidden states
    #     outputs = self.lm_model.model(
    #         input_ids=input_ids,
    #         attention_mask=attention_mask,
    #         output_hidden_states=False
    #     )

    #     # Last hidden state
    #     last_hidden = outputs.last_hidden_state  # [batch, seq_len, hidden_dim]
    #     pooled = last_hidden[:, -1, :]  # last token

    #     # Classification head
    #     logits = self.lm_model.lm_head(pooled)

    #     loss = None
    #     if labels is not None:
    #         loss_fn = nn.CrossEntropyLoss()
    #         loss = loss_fn(logits.view(-1, self.num_labels), labels.view(-1))

    #     if not return_dict:
    #         return (logits,) if loss is None else (loss, logits)

    #     print('the loss and logits are' ,loss , logits)

    #     return {"loss": loss, "logits": logits}

    def forward(self,  input_ids=None, attention_mask=None, labels = None, return_dict =True,**kwargs):
        lm_out = self.lm_model.model(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        last_hidden = lm_out.last_hidden_state         # [B, L, H]
        pooled = last_hidden[:, -1, :]                 # [B, H]
        rewards = self.classifier(pooled)              # [B, 1]

        # return SequenceClassifierOutput with logits=reward
        if not return_dict:
            return (rewards,)
        return SequenceClassifierOutput(logits=rewards)

    def save_pretrained(self, save_directory, **kwargs):
        os.makedirs(save_directory, exist_ok=True)
        torch.save(self.lm_model.lm_head.state_dict(), os.path.join(save_directory, "lm_head.pt"))


In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
max_length = 512

dataset = load_dataset(path ='Dahoas/rm-static', cache_dir = FILE_PATH)


def prep_batch(batch):
    chosen = [p + " " + c for p, c in zip(batch["prompt"], batch["chosen"])]
    rejected = [p + " " + r for p, r in zip(batch["prompt"], batch["rejected"])]
    chosen_tok = tokenizer(chosen, truncation=True, padding="max_length", max_length=max_length)
    rejected_tok = tokenizer(rejected, truncation=True, padding="max_length", max_length=max_length)
    return {
        "chosen_input_ids": chosen_tok["input_ids"],
        "chosen_attention_mask": chosen_tok["attention_mask"],
        "rejected_input_ids": rejected_tok["input_ids"],
        "rejected_attention_mask": rejected_tok["attention_mask"],
    }

tokenized = dataset.map(prep_batch, batched=True, remove_columns=dataset["train"].column_names)


import torch

def simple_reward_collator(batch):
    # batch: list of dicts (from dataset)
    # assume lists of ints for each key
    chosen_ids = torch.tensor([ex["chosen_input_ids"] for ex in batch], dtype=torch.long)
    chosen_mask = torch.tensor([ex["chosen_attention_mask"] for ex in batch], dtype=torch.long)
    rejected_ids = torch.tensor([ex["rejected_input_ids"] for ex in batch], dtype=torch.long)
    rejected_mask = torch.tensor([ex["rejected_attention_mask"] for ex in batch], dtype=torch.long)

    return {
        "chosen_input_ids": chosen_ids,
        "chosen_attention_mask": chosen_mask,
        "rejected_input_ids": rejected_ids,
        "rejected_attention_mask": rejected_mask,
    }


In [ ]:
print(tokenized["train"].column_names)
# must include the four keys used by collator


In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(BASE_MODEL, num_labels=1, cache_dir = FILE_PATH)

# print("config is ", config)

device_str = 'cpu'
conv_model = ConvertedModel(config).to(device_str)

# use trainer class /  reward model in this 
from transformers import HfArgumentParser
from datasets import load_dataset
from trl import (
    ModelConfig,
    RewardConfig,
    RewardTrainer,
    ScriptArguments,
    get_kbit_device_map,
    get_peft_config,
    get_quantization_config,
    setup_chat_format,
)


# dataset.to(device_str)


reward_args = RewardConfig(
    output_dir=os.path.join(FILE_PATH, "reward_outputs"),   # checkpoint + logs dir
    logging_dir=os.path.join(FILE_PATH, "logs"),            # local tensorboard logs
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=2,
    num_train_epochs=10,
    logging_steps=10,
    gradient_checkpointing=False, 
    learning_rate = 0.0000001,
    max_grad_norm = 1.0,

    # Precision
    fp16=False,          # disable mixed precision
    bf16=False,

    # Checkpoint saving
    save_strategy="steps",   # instead of "epoch"
    save_steps=5,            # save every 5 steps
    save_total_limit=2,      # keep only last 2 checkpoints

    # W&B integration
    report_to=[],       # logs training metrics to wandb
    run_name="reward_model_gemma_linear_head"  # nice display name in wandb
)

trainer = RewardTrainer(
    model=conv_model,
    processing_class=None,
    data_collator=simple_reward_collator,
    args=reward_args,
    train_dataset=tokenized['train'], 
    eval_dataset = tokenized['test'],

)


print("Starting the trainer module !!")
trainer.train()


debuggings


In [ ]:
import torch
conv_model = ConvertedModel(config).to('mps')


for n, p in conv_model.named_parameters():
    print(n, p.device, p.dtype, torch.isfinite(p).all().item())


In [ ]:
conv_model.eval()
ids = torch.tensor([tokenizer["train"]["chosen_input_ids"][0]], device='mps')
mask = torch.tensor([tokenizer["train"]["chosen_attention_mask"][0]], device='mps')

with torch.no_grad():
    out = conv_model(input_ids=ids, attention_mask=mask, return_dict=True)
    print("logits:", out.logits, "finite:", torch.isfinite(out.logits).all().item())


chatgpt starts

In [ ]:
# Single runnable cell — minimal, targeted fixes only (copy-paste)
import os
import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoConfig,
    PreTrainedModel,
)
from transformers.modeling_outputs import SequenceClassifierOutput
from trl import RewardConfig, RewardTrainer

# ------------------ USER-CHOICE (preserved) ------------------
BASE_MODEL = "google/gemma-3-270m-it"

# keep your FILE_PATH variable as-is in your environment
# if it's not defined, set a sensible default (uncomment next line)
# FILE_PATH = "./models"
# -------------------------------------------------------------

# 1) tokenizer (used by RewardTrainer as processing_class)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, cache_dir=FILE_PATH)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 2) ConvertedModel: PreTrainedModel wrapper that uses frozen causal LM backbone
class ConvertedModel(PreTrainedModel):
    def __init__(self, config, **kwargs):
        super().__init__(config)
        # preserve your supplied model name if provided; else fall back to config
        self.model_name = kwargs.get("model", getattr(config, "name_or_path", BASE_MODEL))
        # reward head => single scalar
        self.num_labels = kwargs.get("num_labels", getattr(config, "num_labels", 1))

        # load frozen LM backbone
        self.lm_model = AutoModelForCausalLM.from_pretrained(self.model_name, cache_dir=FILE_PATH)
        for p in self.lm_model.parameters():
            p.requires_grad = False

        # replace lm_head with scalar reward head — do NOT pass device here
        in_features = self.lm_model.lm_head.in_features
        self.lm_model.lm_head = nn.Linear(in_features, 1)
        self.classifier = self.lm_model.lm_head
        for p in self.classifier.parameters():
            p.requires_grad = True

    def forward(self, input_ids=None, attention_mask=None, return_dict: bool = True, **kwargs):
        # pass through LM body
        lm_out = self.lm_model.model(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        last_hidden = lm_out.last_hidden_state        # [B, L, H]
        pooled = last_hidden[:, -1, :]                # last token pooling -> [B, H]
        rewards = self.classifier(pooled)             # [B, 1]
        if not return_dict:
            return (rewards,)
        return SequenceClassifierOutput(logits=rewards)

    # lightweight saving: store only head weights (as you requested earlier)
    def save_pretrained(self, save_directory, **kwargs):
        os.makedirs(save_directory, exist_ok=True)
        torch.save(self.classifier.state_dict(), os.path.join(save_directory, "lm_head.pt"))

# 3) build model from config and move to device (CPU by default)
config = AutoConfig.from_pretrained(BASE_MODEL, num_labels=1, cache_dir=FILE_PATH)
conv_model = ConvertedModel(config, model=BASE_MODEL, num_labels=1)

device = torch.device("mps")  # change to "cuda" if you have CUDA and want to use it
conv_model.to(device)

# 4) load raw preference dataset (let RewardTrainer handle tokenization)
dataset = load_dataset("Dahoas/rm-static", cache_dir=FILE_PATH)

# 5) reward/trainer args (keeps your preferences)
reward_args = RewardConfig(
    output_dir=os.path.join(FILE_PATH, "reward_outputs"),
    logging_dir=os.path.join(FILE_PATH, "logs"),
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=32,
    num_train_epochs=10,
    logging_steps=20,
    gradient_checkpointing=False,
    learning_rate=1e-5,
    max_grad_norm=1.0,
    fp16=False,
    bf16=False,
    save_strategy="steps",
    save_steps=5,
    save_total_limit=2,
    report_to=['wandb'],   # no live wandb streaming
    run_name="reward_model_gemma_linear_head",
    max_length=512,
    dataset_num_proc=1,
)

# 6) sanity check: forward with tokenizer on a single example (ensure finite logits)
conv_model.eval()
sample = dataset["train"][0]
# create one prompt+response string (prompt+chosen) to check tokenization + forward
chosen_text = sample["prompt"] + " " + sample["chosen"]
tok = tokenizer(chosen_text, truncation=True, padding="max_length", max_length=512, return_tensors="pt")
inp_ids = tok["input_ids"].to(device)
attn = tok["attention_mask"].to(device)
with torch.no_grad():
    out = conv_model(input_ids=inp_ids, attention_mask=attn, return_dict=True).logits
print("Sanity check logits shape:", out.shape, "finite:", torch.isfinite(out).all().item())

# If logits are not finite, reinitialize head and re-run sanity check
if not torch.isfinite(out).all().item():
    print("Reinitializing head (non-finite detected).")
    in_f = conv_model.classifier.in_features
    conv_model.classifier = nn.Linear(in_f, 1)
    nn.init.normal_(conv_model.classifier.weight, std=0.02)
    nn.init.zeros_(conv_model.classifier.bias)
    for p in conv_model.classifier.parameters(): p.requires_grad = True
    conv_model.to(device)
    with torch.no_grad():
        out = conv_model(input_ids=inp_ids, attention_mask=attn, return_dict=True).logits
    print("After reinit finite:", torch.isfinite(out).all().item())

# 7) instantiate RewardTrainer and run training (let TRL do tokenization)
trainer = RewardTrainer(
    model=conv_model,
    processing_class=tokenizer,   # let TRL create input_ids_chosen / input_ids_rejected
    args=reward_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)

print("Starting training — trainer.train() ...")
trainer.train()


Sanity check logits shape: torch.Size([1, 1]) finite: True


/Users/mohitdulani/Desktop/personal/rlhf/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
socket.send() raised exception.
socket.send() raised exception.
You're using a GemmaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.


Starting training — trainer.train() ...


Step,Training Loss
20,0.965100
40,1.016600


socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.


KeyboardInterrupt: 

socket.send() raised exception.
socket.send() raised exception.


: 